In [1]:
import cv2
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from mediapipe.framework.formats import landmark_pb2
import mediapipe as mp

# Конфигурации

In [2]:
def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    mp.solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    mp.solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

In [3]:
model_path = "face_landmarker.task"

In [4]:
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)

detector = FaceLandmarker.create_from_options(options)

W20230915 13:46:47.543543 15965712 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Тестирование в реальном времени постороение точек на лице

In [5]:
cap = cv2.VideoCapture(0)
while True:
    ret, image = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    face_landmarker_result = detector.detect(mp_image)
    annotated_image = draw_landmarks_on_image(image, face_landmarker_result)

    cv2.imshow('frame', annotated_image)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Создание датаеста с эмоциями

In [5]:
directory_image_path = "emotion-dataset"

emotion = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
emotion_cords_dataset = list()
for dir_emotion in emotion:
    dir_emotion_path = f"{directory_image_path}/{dir_emotion}"
    
    file_names = [file for file in os.listdir(dir_emotion_path) if os.path.isfile(os.path.join(dir_emotion_path, file))]

    data_cord_emotion = list()

    for image in file_names:
        img = cv2.imread(f"{dir_emotion_path}/{image}")
        
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
        face_landmarker_result = detector.detect(mp_image)
        if len(face_landmarker_result.face_landmarks) == 0:
            continue
        face_landmarker_result = face_landmarker_result.face_landmarks[0]
        
        cords_image = list(map(lambda landmark_xyz: [landmark_xyz.x, landmark_xyz.y, landmark_xyz.z], face_landmarker_result))

        data_cord_emotion.append(cords_image)
    emotion_cords_dataset.append(data_cord_emotion)
    
    print(f'completed emotion:{dir_emotion}')
print('Completed all emotion')

completed emotion:angry
completed emotion:disgust
completed emotion:fear
completed emotion:happy
completed emotion:neutral
completed emotion:sad
completed emotion:surprise
Completed all emotion


In [6]:
save_emotion_cords_dataset = emotion_cords_dataset.copy()

# Сохраниение numpy с [460, 460, 460, 460, 460, 460, 460]

In [57]:
[len(i) for i in save_emotion_cords_dataset]

[4187, 460, 4349, 8503, 5854, 4994, 3667]

In [58]:
limited_emotion_cords_dataset = list()
for enotion in save_emotion_cords_dataset:
    
    enotion_copy = enotion.copy()
    np.random.shuffle(enotion_copy)

    limited_emotion_cords_dataset.append(enotion_copy[:460])

In [59]:
[len(i) for i in limited_emotion_cords_dataset]

[478, 478, 478, 478, 478, 478, 478]

In [60]:
limited_emotion_cords_dataset = np.array(limited_emotion_cords_dataset)

In [61]:
np.save('emotion_cords_dataset.npy', np.array(limited_emotion_cords_dataset))

# Сохраниение list с [1000, 460, 1000, 1000, 1000, 1000, 1000]

In [7]:
save_emotion_cords_dataset = emotion_cords_dataset.copy()

In [8]:
[len(i) for i in save_emotion_cords_dataset]

[4187, 460, 4349, 8503, 5854, 4994, 3667]

In [9]:
limited_emotion_cords_dataset = list()
for enotion in save_emotion_cords_dataset:
    
    enotion_copy = enotion.copy()
    np.random.shuffle(enotion_copy)

    limited_emotion_cords_dataset.append(enotion_copy[:1000])

In [10]:
[len(i) for i in limited_emotion_cords_dataset]

[1000, 460, 1000, 1000, 1000, 1000, 1000]

In [11]:
with open('emotion_cords_dataset.pkl', 'wb') as file:
    pickle.dump(limited_emotion_cords_dataset, file)